### Read in word list

In [1]:
import src.utils.git as gitutil
bpath = gitutil.get_root()

In [2]:
word_list_name = 'twl06.txt'

word_list_file = bpath + '/data/processed/' + word_list_name

with open(word_list_file, 'r') as f:
    word_list = f.read().splitlines()

### Initialise

In [3]:
possible_words = word_list

### Read in pre-calculated full cross-check

In [4]:
import numpy as np

In [5]:
full_cross_check = np.load(bpath + '/data/processed/full_cross_check.npy')

### Retrieve diagonals to get letters in correct positions

In [6]:
diags = np.diagonal(full_cross_check, axis1=2, axis2=3)

### Determine remaining letters in incorrect positions

Take row maxes excluding diagonals

In [7]:
rowmaxes = np.logical_and((np.tril(full_cross_check, -1) + np.triu(full_cross_check, 1)).max(axis=3), ~diags)

### Concatenate diag and rowmax outputs to find unique combinations

In [8]:
diag_rowmax = np.concatenate((diags, rowmaxes), axis=2)

Encode 10 element boolean vector as binary number

In [9]:
diag_rowmax_bin_encode = np.matmul(diag_rowmax, np.array([2**i for i in range(9, -1, -1)]))

### Count frequencies of outcomes

In [10]:
from collections import Counter
from math import log

def calc_entropy(diag_rowmax_bin_encode, element, log_base):

    # Determine number of possible remaining words
    num_poss_remaining = len(diag_rowmax_bin_encode[element])

    # Count frequency of each binary encoded outcome
    counts = Counter(diag_rowmax_bin_encode[element])

    # Calculate expected entropy
    entropy = sum([(tup[1]/num_poss_remaining) * log(1/tup[1],log_base) for tup in list(counts.items())])

    return(entropy)

In [11]:
entropies = [calc_entropy(diag_rowmax_bin_encode, i, 2) for i in range(len(diag_rowmax_bin_encode))]

In [12]:
max(entropies)

-6.911970966408277

In [13]:
possible_words[entropies.index(max(entropies))]

'tares'